In [1]:
import torch
torch.__version__

'1.12.1+cu116'

In [2]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-t__mtbzz
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-t__mtbzz
  Resolved https://github.com/facebookresearch/detectron2.git to commit 864913f0e57e87a75c8cc0c7d79ecbd774fc669b
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.1/435.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for detectron2: filename=detectron2-0.

In [6]:
#!unzip images_data.zip

In [7]:
import os
import numpy as np
from PIL import Image
import torch
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.structures import Boxes
import pickle

def load_configuration(output_dir):
    with open(output_dir + "cfg.pickle", "rb") as f:
        return pickle.load(f)
    
margin = 50

# Set up the model and predictor
output_dir = "output/obj_train/"
cfg = load_configuration(output_dir)
cfg.MODEL.WEIGHTS = output_dir + "model_best.pth"
cfg.MODEL.DEVICE = "cpu"
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.70
predictor = DefaultPredictor(cfg)

# Directory containing the dataset images
dataset_dir = "images_data/"

# Directory to save cropped objects
save_dir = "output_crops"
os.makedirs(save_dir, exist_ok=True)

# List of image files in the dataset directory
image_files = [f for f in os.listdir(dataset_dir) if f.endswith((".jpg", ".png"))]

for image_file in image_files:
    # Load the image
    image_path = os.path.join(dataset_dir, image_file)
    image = Image.open(image_path)
    image_array = np.array(image)

    # Perform inference
    outputs = predictor(image_array[:, :, ::-1])
    instances = outputs["instances"]

    # Crop and save detected objects
    boxes = instances.pred_boxes if isinstance(instances.pred_boxes, Boxes) else instances.pred_boxes.tensor
    for i, bbox in enumerate(boxes):
        x1, y1, x2, y2 = bbox.cpu().numpy().astype(int)
        
        # Ajustar las coordenadas del bounding box con el margen
        x1 -= margin
        y1 -= margin
        x2 += margin
        y2 += margin

        # Asegurarse de que las coordenadas no excedan los límites de la imagen
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(image_array.shape[1], x2)
        y2 = min(image_array.shape[0], y2)

        
        cropped_image = image_array[y1:y2, x1:x2]
        cropped_image = Image.fromarray(cropped_image)
        save_path = os.path.join(save_dir, f"{os.path.splitext(image_file)[0]}_object_{i}.jpg")
        cropped_image.save(save_path)

/usr/local/lib/python3.9/dist-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


## Make a Zip file
We save the file in ZIP format in case it is needed for later use in other applications.

In [9]:
#!zip -r crop_images.zip output_crops/